In [ ]:
import tensorflow as tf
from keras import layers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import time


PROJECT_DIR = ".."
DATA_DIR = f"{PROJECT_DIR}/data/processed/PlantVillage_Binary"
LOGS_DIR = f"{PROJECT_DIR}/logs/disease_detection"

# Finetuning parameters
BATCH_SIZE = 32
IMG_SIDE_LENGTH = 128
IMG_SIZE = (IMG_SIDE_LENGTH, IMG_SIDE_LENGTH)
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE = 2
EPOCHS = 30
LR = 1e-4
LABEL_MODE = 'binary'

CLASS_WEIGHTS = None # Not required because both classes have equal images


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU memory growth set to True")
    except RuntimeError as e:
        print(e)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=f"{DATA_DIR}/train",
    labels="inferred",
    label_mode=LABEL_MODE,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATA_DIR}/val",
    labels="inferred",
    label_mode=LABEL_MODE,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    f"{DATA_DIR}/test",
    labels="inferred",
    label_mode=LABEL_MODE,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
class_names = train_ds.class_names

print("Class names:", class_names)
print("Class 0 maps to:", class_names[0])
print("Class 1 maps to:", class_names[1])

In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"), 
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.3),
    layers.RandomContrast(0.3),
    layers.RandomTranslation(0.1, 0.1),
    # Add noise
    layers.GaussianNoise(0.1),     
])

In [ ]:
# Visualize the augmented data with labels
import numpy as np

plt.figure(figsize=(15, 10))

# Get a batch of data
for images, labels in train_ds.take(1):
    # Take first 8 images from the batch
    for i in range(8):
        # Original image
        plt.subplot(4, 4, i*2 + 1)
        # Denormalize for display (since normalization_layer is applied in model)
        original_img = images[i].numpy().astype("uint8")
        plt.imshow(original_img)
        plt.title(f'Original: {class_names[int(labels[i])]}')
        plt.axis('off')
        
        # Augmented image
        plt.subplot(4, 4, i*2 + 2)
        # Apply data augmentation
        augmented_img = data_augmentation(tf.expand_dims(images[i], 0), training=True)
        augmented_img = tf.cast(augmented_img[0], tf.uint8).numpy()
        plt.imshow(augmented_img)
        plt.title(f'Augmented: {class_names[int(labels[i])]}')
        plt.axis('off')
    break

plt.tight_layout()
plt.show()

# Show distribution of classes in the current batch
unique, counts = np.unique(labels.numpy(), return_counts=True)
print(f"\nClass distribution in this batch:")
for class_idx, count in zip(unique, counts):
    print(f"{class_names[int(class_idx)]}: {count} images")

In [ ]:
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=BUFFER_SIZE)
val_ds = val_ds.prefetch(buffer_size=BUFFER_SIZE)
test_ds = test_ds.prefetch(buffer_size=BUFFER_SIZE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIDE_LENGTH, IMG_SIDE_LENGTH, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # freeze base

model = tf.keras.Sequential([
    normalization_layer,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(IMG_SIDE_LENGTH, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
model.compile(optimizer=Adam(learning_rate=LR),
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])

In [ ]:
# Define callbacks for better training control
import os

# Create models directory if it doesn't exist
os.makedirs(f"{PROJECT_DIR}/models", exist_ok=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
]

In [ ]:
# Start timing
training_start_time = time.time()
print("Starting training...")

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    class_weight=CLASS_WEIGHTS,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
# End timing
training_end_time = time.time()
print(f"Training completed in {training_end_time - training_start_time:.2f} seconds")

In [ ]:
# Minimal improvements to existing plots
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='val accuracy', linewidth=2)
plt.title('Accuracy', fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train loss', linewidth=2)
plt.plot(history.history['val_loss'], label='val loss', linewidth=2)
plt.title('Loss', fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['precision'], label='train precision', linewidth=2)
plt.plot(history.history['val_precision'], label='val precision', linewidth=2)
plt.title('Precision', fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history.history['recall'], label='train recall', linewidth=2)
plt.plot(history.history['val_recall'], label='val recall', linewidth=2)
plt.title('Recall', fontsize=14)
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Train Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
# Evaluate the model on the test dataset
test_results = model.evaluate(test_ds)
test_loss = test_results[0]
test_accuracy = test_results[1]
test_precision = test_results[2] if len(test_results) > 2 else None
test_recall = test_results[3] if len(test_results) > 3 else None

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
if test_precision is not None:
    print(f"Test Precision: {test_precision}")
if test_recall is not None:
    print(f"Test Recall: {test_recall}")

In [ ]:
# Load and preprocess your custom image
from keras.utils import load_img, img_to_array
import numpy as np

# Path to your custom image
custom_image_path_healthy = f"../data/custom/image.png" 
custom_image_path_diseased = f"../data/custom/bp_diseased.jpg" 

diseased_image = load_img(custom_image_path_diseased, target_size=IMG_SIZE)
diseased_image_array = img_to_array(diseased_image)
diseased_image_array = np.expand_dims(diseased_image_array, axis=0)
diseased_image_array = diseased_image_array / 255.0

healthy_image = load_img(custom_image_path_healthy, target_size=IMG_SIZE)
healthy_image_array = img_to_array(healthy_image)
healthy_image_array = np.expand_dims(healthy_image_array, axis=0)
healthy_image_array = healthy_image_array / 255.0

healthy_predictions = model.predict(healthy_image_array)
diseased_predictions = model.predict(diseased_image_array)

print("Raw predictions:")
print(f"Healthy image raw score: {healthy_predictions[0][0]:.4f}")
print(f"Diseased image raw score: {diseased_predictions[0][0]:.4f}")
print()

predicted_healthy_class = (healthy_predictions[0][0] > 0.1).astype(int)
predicted_diseased_class = (diseased_predictions[0][0] > 0.1).astype(int)

print(f"Healthy Image Prediction: {class_names[predicted_healthy_class]} (Score: {healthy_predictions[0][0]:.4f})")
print(f"Diseased Image Prediction: {class_names[predicted_diseased_class]} (Score: {diseased_predictions[0][0]:.4f})")

In [ ]:
# Metrics
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np

# Get predictions for test set
y_pred_probs = model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Get true labels
y_true = []
for _, labels in test_ds:
    y_true.extend(labels.numpy())
y_true = np.array(y_true)

# Calculate metrics
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Training Run Logger
import json
import datetime
import os

# Create logs directory if it doesn't exist
os.makedirs(LOGS_DIR, exist_ok=True)

# Calculate training time (assuming you stored start time before training)
# If you didn't store start time, you can estimate from history length
try:
    training_time_seconds = training_end_time - training_start_time
except NameError:
    # Estimate based on epochs (rough approximation)
    epochs_completed = len(history.history['accuracy'])
    estimated_time_per_epoch = 60  # seconds (adjust based on your setup)
    training_time_seconds = epochs_completed * estimated_time_per_epoch

training_time_minutes = training_time_seconds / 60
training_time_hours = training_time_minutes / 60

# Prepare training log data
training_log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "hyperparameters": {
        "batch_size": BATCH_SIZE,
        "img_side_length": IMG_SIDE_LENGTH,
        "epochs": EPOCHS,
        "learning_rate": LR,
        "buffer_size": BUFFER_SIZE,
        "label_mode": LABEL_MODE,
        "class_weights": CLASS_WEIGHTS,
        "data_augmentation": "RandomFlip(horizontal) + RandomRotation(0.5)",
        "base_model": "MobileNetV2",
        "base_model_trainable": False,
        "optimizer": "Adam",
        "loss": "binary_crossentropy",
        "dropout_rate": 0.3,
        "dense_layer_units": IMG_SIDE_LENGTH
    },
    "dataset_info": {
        "data_dir": DATA_DIR,
        "class_names": class_names,
        "total_classes": len(class_names)
    },
    "callbacks": {
        "early_stopping": {
            "monitor": "val_loss",
            "patience": 5,
            "restore_best_weights": True
        },
        "reduce_lr_on_plateau": {
            "monitor": "val_loss",
            "factor": 0.5,
            "patience": 3,
            "min_lr": 1e-7
        }
    },
    "training_time": {
        "total_seconds": float(training_time_seconds),
        "total_minutes": float(training_time_minutes),
        "total_hours": float(training_time_hours),
        "formatted": f"{int(training_time_hours):02d}h {int(training_time_minutes % 60):02d}m {int(training_time_seconds % 60):02d}s"
    },
    "results": {
        "final_train_accuracy": float(history.history['accuracy'][-1]),
        "final_val_accuracy": float(history.history['val_accuracy'][-1]),
        "final_train_loss": float(history.history['loss'][-1]),
        "final_val_loss": float(history.history['val_loss'][-1]),
        "best_val_accuracy": float(max(history.history['val_accuracy'])),
        "best_val_loss": float(min(history.history['val_loss'])),
        "epochs_trained": len(history.history['accuracy']),
        "test_accuracy": float(test_accuracy),
        "test_loss": float(test_loss),
        "test_f1_score": float(f1),
        "test_precision": float(precision),
        "test_recall": float(recall)    
    },
    "training_history": {
        "accuracy": [float(x) for x in history.history['accuracy']],
        "val_accuracy": [float(x) for x in history.history['val_accuracy']],
        "loss": [float(x) for x in history.history['loss']],
        "val_loss": [float(x) for x in history.history['val_loss']]
    },
    "custom_predictions": {
        "healthy_image_score": float(healthy_predictions[0][0]),
        "diseased_image_score": float(diseased_predictions[0][0]),
        "healthy_image_prediction": class_names[predicted_healthy_class],
        "diseased_image_prediction": class_names[predicted_diseased_class]
    }
}

# Save to JSON file with timestamp
timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"{LOGS_DIR}/training_log_{timestamp_str}.json"

with open(log_filename, 'w') as f:
    json.dump(training_log, f, indent=2)

print(f"Training log saved to: {log_filename}")

# Print summary
print("\n=== TRAINING SUMMARY ===")
print(f"Model: MobileNetV2 (frozen) + Dense({IMG_SIDE_LENGTH}) + Dense(1)")
print(f"Training Time: {training_log['training_time']['formatted']}")
print(f"Final Training Accuracy: {training_log['results']['final_train_accuracy']:.4f}")
print(f"Final Validation Accuracy: {training_log['results']['final_val_accuracy']:.4f}")
print(f"Test Accuracy: {training_log['results']['test_accuracy']:.4f}")
print(f"Test F1 Score: {training_log['results']['test_f1_score']:.4f}")
print(f"Epochs Trained: {training_log['results']['epochs_trained']}")
print(f"Best Validation Accuracy: {training_log['results']['best_val_accuracy']:.4f}")
print(f"Learning Rate: {LR}")
print(f"Batch Size: {BATCH_SIZE}")

In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open(f"{PROJECT_DIR}/models/plant_disease__binary_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite format!")